In [81]:
import numpy as np

import torch
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [82]:
train_data = MNIST(root="./", train=True, transform=None, target_transform=None, download=True)
test_data = MNIST(root="./", train=False, transform=None, target_transform=None, download=True)


In [83]:
conv = torch.nn.Conv2d(1,1,3,1,0)
pool = torch.nn.MaxPool2d(2)
input = torch.zeros(2, 1, 28, 28)
out = conv.forward(input)
out = pool.forward(out)

print(out.shape)


torch.Size([2, 1, 13, 13])


In [90]:
class simple_cnn(torch.nn.Module):
    def __init__(self):
        super(simple_cnn, self).__init__()

        self.conv = torch.nn.Conv2d(1, 1, 3, 1, 0)
        self.relu = torch.nn.ReLU()
        self.pool = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(13**2, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.flatten(1)
        x = self.fc(x)
        x = self.softmax(x)

        return x

In [85]:
cnn = simple_cnn()
input = train_data.__getitem__(0)[0]
input = torch.tensor(np.array(input)).unsqueeze(0).unsqueeze(0).float()
out = cnn(input)



/home/beckmann/miniconda3/envs/deepfake/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [86]:
def label_one_hot_encoding(x, dim=10):
    output = np.zeros(dim)
    output[x] = 1
    return output

In [96]:
cnn = simple_cnn().to("cpu")
batch_size = 4
learning_rate = 1e-1

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=learning_rate)

num_samples = len(train_data)
num_epochs = 5

for i in range(num_epochs*num_samples // batch_size):
    input = np.zeros((batch_size, 28, 28))
    targets = np.zeros((batch_size, 10))

    for j in range(batch_size):
        # load img
        index = (i*batch_size + j) % (num_samples-1)

        img = np.array(train_data.__getitem__(index)[0]) / 255.
        input[j] = img

        # load labels
        label = np.array(train_data.__getitem__(index)[1])
        label = label_one_hot_encoding(label)
        targets[j] = label

    input = torch.tensor(input).float().unsqueeze(1).to("cpu")
    targets = torch.tensor(targets).float().to("cpu")

    pred = cnn.forward(input)

    loss = loss_fn(targets, pred)

    print(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/home/beckmann/miniconda3/envs/deepfake/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


tensor(0.0891, grad_fn=<MseLossBackward0>)
tensor(0.0914, grad_fn=<MseLossBackward0>)
tensor(0.0881, grad_fn=<MseLossBackward0>)
tensor(0.0911, grad_fn=<MseLossBackward0>)
tensor(0.0917, grad_fn=<MseLossBackward0>)
tensor(0.0916, grad_fn=<MseLossBackward0>)
tensor(0.0881, grad_fn=<MseLossBackward0>)
tensor(0.0894, grad_fn=<MseLossBackward0>)
tensor(0.0929, grad_fn=<MseLossBackward0>)
tensor(0.0932, grad_fn=<MseLossBackward0>)
tensor(0.0902, grad_fn=<MseLossBackward0>)
tensor(0.0911, grad_fn=<MseLossBackward0>)
tensor(0.0936, grad_fn=<MseLossBackward0>)
tensor(0.0897, grad_fn=<MseLossBackward0>)
tensor(0.0910, grad_fn=<MseLossBackward0>)
tensor(0.0897, grad_fn=<MseLossBackward0>)
tensor(0.0881, grad_fn=<MseLossBackward0>)
tensor(0.0914, grad_fn=<MseLossBackward0>)
tensor(0.0919, grad_fn=<MseLossBackward0>)
tensor(0.0883, grad_fn=<MseLossBackward0>)
tensor(0.0941, grad_fn=<MseLossBackward0>)
tensor(0.0917, grad_fn=<MseLossBackward0>)
tensor(0.0901, grad_fn=<MseLossBackward0>)
tensor(0.09

In [97]:
cnn.eval()

num_test_samples = len(test_data)
results = []

for i in range(num_test_samples):
    input = np.array(test_data.__getitem__(i)[0]) / 255.
    input = torch.tensor(input).float().unsqueeze(0).unsqueeze(0)

    label = np.array(test_data.__getitem__(i)[1])
    label = label_one_hot_encoding(label)
    label = torch.tensor(label).float()

    pred = cnn.forward(input)

    if pred.argmax() == label.argmax():
        results.append(1)
    else:
        results.append(0)

print(np.mean(results))

/home/beckmann/miniconda3/envs/deepfake/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


0.9099
